Copyright 2023 Bumghi Choi.

In [1]:
# 이 프로그램은 코스피200선물 거래를 위해서 tensorflow 홈페이지에 있는 actor-critic sample을 기반으로 작성되었다.
# 거래 수익률과 현재 보유 상태를 state로 하고 매수/매도 를  action으로 한다.
# actor에 의한 거래 시그널이 과거 60분봉 선물지수 데이터에 의거하여 수익률과 보유 상태의 변화를 준다.
# A2C (advantage actor-critic) 방식으로 구성하고 학습한다.

# Actor-Critic 메서드로 코스피200선물 거래하기 - DeepMoney

**Actor-Critic 방법**

Actor-Critic 방법은 가치 함수와 독립적인 정책 함수를 나타내는 [Temporal Difference(TD) 학습](https://en.wikipedia.org/wiki/Temporal_difference_learning) 방법입니다.

정책 함수(또는 정책)는 에이전트가 주어진 상태에 따라 취할 수 있는 동작에 대한 확률 분포를 반환합니다. 가치 함수는 주어진 상태에서 시작하여 특정 정책에 따라 영원히 동작하는 에이전트의 예상 이익을 결정합니다.

Actor-Critic 방법에서 정책은 주어진 상태에 따라 가능한 일련의 동작을 제안하는 *행위자*라고 하며, 추정값 함수는 주어진 정책에 따라 *행위자*가 취한 동작을 평가하는 *비평가*라고 합니다.

이 튜토리얼에서 *행위자*와 *비평가* 모두 두 개의 출력이 있는 하나의 신경망을 사용하여 표현됩니다.


코스피 200 선물 - 60분봉 데이터 - actor에 의한 거래 시그널을 적용하여 거래한 결과 (예)

* 0 : 중립,  1: 매도,   2: 매수

date	거래 시그널	종가	수익	수수료
2022/04/19/13:00	1	357.5	0	0
2022/04/19/14:00	1	357.25	0	0
2022/04/19/15:00	2	357.2	75000	5360.25
2022/04/20/09:00	0	355.7	0	0
2022/04/20/10:00	2	355.45	0	0
2022/04/20/11:00	0	356	0	0
2022/04/20/12:00	0	357.1	0	0
2022/04/20/13:00	0	357.7	0	0
2022/04/20/14:00	0	357.45	500000	5346.75
2022/04/20/15:00	2	356.95	0	0
2022/04/21/09:00	0	359.85	0	0
2022/04/21/10:00	2	360.4	0	0
2022/04/21/11:00	2	360.2	0	0
2022/04/21/12:00	0	359.45	0	0
2022/04/21/13:00	0	359.7	0	0
2022/04/21/14:00	0	359.65	-187500	5400.375
2022/04/21/15:00	2	358.9	0	0
2022/04/22/09:00	1	353.75	0	0
2022/04/22/10:00	1	354.6	0	0


이 문제는 거래 에피소드에 대한 평균 총 수익이 100분봉 거래에서 5* 250000에 도달하면 "해결"된 것으로 간주됩니다.

## 설정

필요한 패키지를 가져오고 전역 설정을 구성합니다.


In [2]:
import collections
import numpy as np
import pandas as pd
import statistics
import tensorflow as tf
import tqdm
import pandas as pd

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

## 코스피200 선물 파일로부터 tran, test 를 위한 dataframe을 만든다.

In [3]:
df0 = pd.read_csv("kospi200f_60M.csv", encoding="euc-kr")
train_df = df0.loc[df0['date'] <= '2021/12/31/15:00']
train_df = train_df.loc[train_df['date'] >= '2017/01/01/09:00']

test_df = df0.loc[df0['date'] >= '2022/01/01/09:00']

## 모델

*행위자*와 *비평가*는 각각 동작 확률과 비평 값을 생성하는 하나의 신경망을 사용하여 모델링됩니다. 이 튜토리얼에서는 모델 하위 클래스화를 사용하여 모델을 정의합니다.

순방향 전달 중에 모델은 상태를 입력으로 받고 상태 종속 값을 모델링하는 동작 확률과 비평 값 $V$를 모두 출력합니다. 목표는 예상 이익을 최대화하는 $\pi$ 정책을 기반으로 행동을 선택하는 모델을 훈련하는 것입니다.

deepmoney의 경우, 상태를 나타내는 4 가지 값이 있는데, 각각 20일 수익률(%), 5일 수익률(%), 1일 수익률(%) 및 보유 상태(0: 없음, 1:매도, 2: 매수)입니다. 에이전트는 중립(0, 거래없음), 매도(1, 청산후 진입)와 매수(2, 청산후 진입) 3가지 동작을 취할 수 있습니다.

In [4]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x), self.critic(x)

In [5]:
num_actions = 3  # 2
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

# 거래 환경 제공 및 훈련 데이터 생성

2022-01-01~ 2023-01-20 까지의 코스피200 선물 가격 데이터로부터 모델의 action(매수/매도 거래 시그널)과 현재 state에 의한 수익을 평가하고 5일, 20일 평균 수익을 계산하여 새로운 state를 반환한다.

In [6]:
class env():
  """Combined actor-critic network."""

  def __init__(
      self, 
      mode: str): 

    """Initialize."""

    self.mode = mode
    self.initial_pos = 0
    self.current_pos = 0
    self.train_df = train_df
    self.test_df = test_df
    if self.mode == 'train':
        self.price = self.train_df['종가'].values
    else:
        self.price = self.test_df['종가'].values
    self.state = tf.constant([0.0,0.0,0.0,0.0,0.0,0], dtype=tf.float32)

  def step(self, action: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:

    if self.mode == 'train' and self.current_pos > len(self.train_df) - 2:
        return self.state, tf.constant(0)
    elif self.mode == 'test' and self.current_pos > len(self.test_df) - 2:
        return self.state, tf.constant(0)
    
    # state(일자별 수익률) 한칸씩 왼쪽으로 이동
    new_state  = np.zeros(6)
    for i in range(5):
        new_state[i] = self.state[i+1]
        
  
    # action에 의한 보유 상태 변경
    if action == 1:
        new_state[5] = max(self.state[5] - 1, -1)
    elif action == 2:
        new_state[5] = min(self.state[5] + 1, 1)    
    
    # 변경된 보유상태에서 가격 변동에 따른 마지막 step의 수익률 변경
    if new_state[5] == 0:
        new_state[4] = 0
    elif new_state[5] == 1:
        new_state[4] = (self.price[self.current_pos+1] - self.price[self.current_pos]) / self.price[i]
    else:
        new_state[4] = (self.price[self.current_pos] - self.price[self.current_pos+1]) / self.price[i]

    # 손익 여부(-1, 0, 1)를 나타내는 rewrad 산출
    reward = tf.sign(new_state[4])
    
    # 현재 position 이동
    self.current_pos += 1

    # 변경된 state 저장
    self.state = tf.convert_to_tensor(new_state)
    
    return self.state, reward

  def reset(self) -> tf.Tensor:
    self.sate = tf.constant([0.0,0.0,0.0,0.0,0.0,0], dtype=tf.float32)
    self.current_pos = self.initial_pos
    return self.state

env = env(0)

## 에이전트 훈련

에이전트를 훈련하기 위해 다음 단계를 따릅니다.

1. 환경에서 에이전트를 실행하여 에피소드별로 훈련 데이터를 수집합니다.
2. 각 시간 스텝에서 예상 이익을 계산합니다.
3. 결합된 Actor-Critic 모델의 손실을 계산합니다.
4. 그래디언트를 계산하고 네트워크 매개변수를 업데이트합니다.
5. 성공 기준 또는 최대 에피소드에 도달할 때까지 1~4를 반복합니다.


### 1. 훈련 데이터 수집

지도 학습에서와 같이 Actor-Critic 모델을 훈련하려면 훈련 데이터가 필요합니다. 그러나, 이러한 데이터를 수집하려면 모델이 환경에서 "실행"되어야 합니다.

여기서는 각 에피소드에 대한 훈련 데이터를 수집합니다. 그런 다음, 모델의 가중치에 의해 매개변수화된 현재 정책을 기반으로 동작 확률과 비평 값을 생성하기 위해 각 타임스텝에서 모델의 순방향 전달을 환경 상태에서 실행합니다.

다음 동작은 모델에 의해 생성된 동작 확률로부터 샘플링되며, 그런 다음 환경에 적용되어 다음 상태와 보상을 생성합니다.

이 프로세스는 더 빠른 훈련을 위해 나중에 TensorFlow 그래프로 컴파일할 수 있도록 TensorFlow 연산을 사용하는 `run_episode` 함수에서 구현됩니다. `tf.TensorArray`는 가변 길이 배열에서 Tensor 반복을 지원하는 데 사용되었습니다.

In [7]:
# initiate env to apply a step of action
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward = env.step(action)
  return (np.array(state, np.float32), 
          np.array(reward, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32])

In [8]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
  
    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])
  
    # Apply action to the environment to get next state and reward
    state, reward = tf_env_step(action)
    state.set_shape(initial_state_shape)
  
    # Store reward
    rewards = rewards.write(t, reward)


  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()
  
  return action_probs, values, rewards

In [11]:
state = env.reset()
a, c, c = run_episode(state, model, 10)

### 2. 예상 이익 계산

한 에피소드 동안 수집된 각 타임스텝 $t$, ${r_{t}}^{T}*{t=1}$에서 보상의 시퀀스를 예상 이익 ${G*{t}}^{T}_{t=1}$의 시퀀스로 변환합니다. 여기서 보상의 합계는 현재 타임스텝 $t$에서 $T$까지 계산되며, 각 보상에 기하급수적으로 감소하는 할인 계수 $\gamma$를 곱합니다.

$$G_{t} = \sum^{T}_{t'=t} \gamma^{t'-t}r_{t'}$$

$\gamma\in(0,1)$ 이후, 현재 타임스텝에서 더 멀리 떨어진 보상에는 더 적은 가중치가 부여됩니다.

직관적으로, 예상 이익은 단순히 지금 보상이 이후 보상보다 낫다는 것을 암시합니다. 이것은 수학적 의미에서 보상의 합이 수렴하도록 하려는 것입니다.

To stabilize training, the resulting sequence of returns is also standardized (i.e. to have zero mean and unit standard deviation).


In [12]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

In [13]:
print(env.reset())
a, b, rewards = run_episode(env.state, model, 30)
print(get_expected_return(rewards, 0.99, True))
print(a)

tf.Tensor([-0.00088585  0.         -0.00113895 -0.00063275  0.          0.        ], shape=(6,), dtype=float64)
tf.Tensor(
[-2.48302    -1.2427393   0.01006951 -1.2361646   0.01671065  0.02639236
  1.2920207   0.05873558  0.06884184  1.334899    1.3578957   0.12527598
  1.391903    1.4154757   0.18343768  0.19480352  0.20628415 -1.0379679
  0.21690923  0.22861314  0.24043532  1.5082256   0.2771244   0.28943658
  0.3018731  -0.94141346 -2.1972585  -0.9540912  -0.9542155   0.3015076 ], shape=(30,), dtype=float32)
tf.Tensor(
[0.3333217  0.3429361  0.34298012 0.34285575 0.35848567 0.3332165
 0.3608474  0.3023004  0.33345374 0.3022401  0.33353767 0.34282923
 0.35851708 0.33329335 0.3022389  0.3333925  0.33333188 0.36091852
 0.33662593 0.33330247 0.33331957 0.36094564 0.33676463 0.33340073
 0.33324492 0.36084527 0.36102754 0.33643493 0.33394197 0.34286952], shape=(30,), dtype=float32)


### 3. Actor-Critic 손실

하이브리드 Actor-Critic 모델을 사용하고 있기 때문에, 아래와 같이 훈련을 위해 Actor와 Critic 손실의 조합인 손실 함수를 사용합니다.

$$L = L_{actor} + L_{critic}$$

#### Actor 손실

[비평가가 상태 종속 기준선인 정책 그래디언트](https://www.youtube.com/watch?v=EKqxumCuAAY&t=62m23s)를 기반으로 행위자 손실을 공식화하고 단일 샘플(에피소드별) 추정치를 계산합니다.

$$L_{actor} = -\sum^{T}_{t=1} \log\pi_{\theta}(a_{t} | s_{t})[G(s_{t}, a_{t})  - V^{\pi}_{\theta}(s_{t})]$$

여기서:

- $T$: 에피소드별로 달라질 수 있는 에피소드별 타임스텝의 수
- $s_{t}$: $t$ 타임스텝의 상태
- $a_{t}$: $s$ 상태에 따라 $t$ 타임스텝에서 선택된 동작
- $\pi_{\theta}$: $\theta$에 의해 매개변수화된 정책(Actor)
- $V^{\pi}_{\theta}$: 마찬가지로 $\theta$에 의해 매개변수화된 값 함수(Critic)
- $G = G_{t}$: 주어진 상태에 대한 예상 이익, 타임스텝 $t$에서 동작 쌍

A negative term is added to the sum since the idea is to maximize the probabilities of actions yielding higher rewards by minimizing the combined loss.

<br>

##### 이점

$L_{actor}$ 공식에서 $G - V$ 항을 [이점](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#advantage-functions)이라고 하며, 이는 특정한 상태에서 $\pi$ 정책에 따라 선택된 임의의 동작보다 이 상태에 얼마나 더 나은 동작이 주어지는지를 나타냅니다.

기준선을 제외할 수 있지만 이로 인해 훈련 중에 큰 변동이 발생할 수 있습니다. 그리고 비평가 $V$를 기준선으로 선택할 때의 좋은 점은 가능한 한 $G$에 가깝게 훈련되어 변동이 낮아진다는 것입니다.

또한, Critic이 없으면 알고리즘이 예상 이익을 바탕으로 특정 상태에서 취하는 행동의 확률을 높이려고 시도할 것이며, 이 때 동작 사이의 상대적 확률이 같게 유지된다면 큰 차이가 생기지 않습니다.

예를 들어, 주어진 상태에서 두 행동의 예상 이익이 같다고 가정합니다. Critic이 없으면 알고리즘은 목표 $J$에 따라 이들 동작의 확률을 높이려고 합니다. Critic의 경우, 이점($G - V = 0$)이 없기 때문에 동작의 확률을 높이는 데 따른 이점이 없으며 알고리즘이 그래디언트를 0으로 설정합니다.

<br>

#### The Critic loss

$V$를 $G$에 최대한 가깝게 훈련하는 것은 다음 손실 함수를 사용한 회귀 문제로 설정할 수 있습니다.

$$L_{critic} = L_{\delta}(G, V^{\pi}_{\theta})$$

여기서 $L_{\delta}$는 [Huber 손실](https://en.wikipedia.org/wiki/Huber_loss)로, 제곱 오차 손실보다 데이터의 이상 값에 덜 민감합니다.


In [14]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

### 4. 매개변수를 업데이트하기 위한 훈련 단계 정의

위의 모든 단계를 모든 에피소드에서 실행되는 훈련 단계로 결합합니다. 손실 함수로 이어지는 모든 단계는 `tf.GradientTape` 컨텍스트로 실행되어 자동 미분이 가능합니다.

이 튜토리얼에서는 Adam 옵티마이저를 사용하여 모델 매개변수에 그래디언트를 적용합니다.

할인되지 않은 보상의 합계인 `episode_reward`도 이 단계에서 계산됩니다. 이 값은 나중에 성공 기준이 충족되는지 평가하는 데 사용됩니다.

`tf.function` 컨텍스트를 `train_step` 함수에 적용하여 호출 가능한 TensorFlow 그래프로 컴파일할 수 있고, 그러면 훈련 속도가 10배 빨라질 수 있습니다.


In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


#@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate the expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

In [16]:
print(env.reset())
print(env.current_pos)

tf.Tensor(
[ 2.78410529e-03 -1.29081245e-02  0.00000000e+00 -1.89825361e-03
  5.06200962e-04  1.00000000e+00], shape=(6,), dtype=float64)
0


### 5. 훈련 루프 실행하기

성공 기준 또는 최대 에피소드 수에 도달할 때까지 훈련 단계를 실행하는 방식으로 훈련을 실행합니다.

대기열을 사용하여 에피소드 보상의 실행 레코드를 유지합니다. 100회 시도에 도달하면 가장 오래된 보상이 대기열의 왼쪽(꼬리쪽) 끝에서 제거되고 최근 보상이 머리쪽(오른쪽)에 추가됩니다. 계산 효율을 높이기 위해 보상의 누적 합계도 유지됩니다.

런타임에 따라 훈련은 1분 이내에 완료될 수 있습니다.

In [17]:
%%time

min_episodes_criterion = 100
max_episodes = 1000
max_steps_per_episode = 500 #100 #500

# `CartPole-v1` is considered solved if average reward is >= 475 over 500 
# consecutive trials
reward_threshold = 475
running_reward = 0

# The discount factor for future rewards
gamma = 0.99

# Keep the last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

t = tqdm.trange(max_episodes)
for i in t:
    initial_state = env.reset()
    initial_state = tf.constant(initial_state, dtype=tf.float64)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))
    
    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)
  

    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)
  
    # Show the average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')
  
    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

100%|██████████████████████████████████████| 1000/1000 [34:28<00:00,  2.07s/it, episode_reward=-1, running_reward=6.67]


Solved at episode 999: average reward: 6.67!
Wall time: 34min 28s


In [18]:
def test(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    """Runs a single episode to collect training data."""

    actions = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)

    initial_state_shape = initial_state.shape
    state = initial_state

    for t in tf.range(max_steps):
        # Convert state into a batched tensor (batch size = 1)
        state = tf.expand_dims(state, 0)

        # Run the model and to get action probabilities and critic value
        action_logits_t, value = model(state)

        # Sample next action from the action probability distribution
        action = tf.random.categorical(action_logits_t, 1)[0, 0]


        # Store actions
        actions = actions.write(t, action)


        # Apply action to the environment to get next state and reward
        state, reward = tf_env_step(action)
        state.set_shape(initial_state_shape)

    actions = actions.stack()
  
    return actions

In [19]:
env.mode = 'test'
initial_state = env.reset()
initial_state = tf.constant(initial_state, dtype=tf.float64)

env.test_df = test_df[:100]
env.price = env.test_df['종가'].values

max_steps = len(env.price) - 1

In [20]:
actions = test(initial_state, model, max_steps)

# 15시 종가를 익일 시가로 조정
#for i in range(len(df)-1):
#    if i != 0 and df.loc[i, 'date'][11:13] == '15':
#        df.loc[i, 'close'] = df.loc[i+1, 'open']

state = 0
count = 0
buy_price = 0
profit = []
fee = []

loss_cut = 0.01
pred_term = 5

for i in range(len(actions) - 1):

    pred = actions[i]
    close = float(test_df['종가'].values[i])
    high = float(test_df['고가'].values[i])
    low = float(test_df['저가'].values[i])
    open = float(test_df['시가'].values[i])
    date = test_df['date'].values[i]

    if date[11:13] == '15':
        if state == 1:
            profit.append((buy_price - close) * 250000)
            fee.append((buy_price + close) * 250000 * 0.00003)
        elif state == 2:
            profit.append((close - buy_price) * 250000)
            fee.append((buy_price + close) * 250000 * 0.00003)
        else:
            profit.append(0)
            fee.append(0)

        state = 0
        count = 0
        buy_price = 0
        
        continue

    if state == 1:

        if high - buy_price >= buy_price * loss_cut:
            profit.append(-int((buy_price*loss_cut+0.05)/0.05)*0.05*250000)
            fee.append((buy_price + close)*250000*0.00003)
            state = pred
            count = 1
            buy_price = close
        elif pred == 2:
            profit.append((buy_price - close)*250000)
            fee.append((buy_price + close)*250000*0.00003)
            state = pred
            count = 1
            buy_price = close
        else:
            profit.append(0)
            fee.append(0)
            count += 1
            count %= pred_term
    elif state == 2:

        if buy_price - low >= buy_price * loss_cut:
            profit.append(-int((buy_price*loss_cut+0.05)/0.05)*0.05*250000)
            fee.append((buy_price + close)*250000*0.00003)
            state = pred
            count = 1
            buy_price = close
        elif pred == 1:
            profit.append((close - buy_price)*250000)
            fee.append((buy_price + close)*250000*0.00003)
            state = pred
            count = 1
            buy_price = close
        else:
            profit.append(0)
            fee.append(0)
            count += 1
            count %= pred_term
    else:
        if pred == 1 or pred == 2:
            state = pred
            count = 1
            buy_price = close
        else:
            count = 0
        profit.append(0)
        fee.append(0)

print((sum(profit) - sum(fee))/test_df['종가'].values.mean()/1.25/250000/0.075 + 1)
    

1.3952532734198648


## 다음 단계

이 튜토리얼에서는 Tensorflow를 사용하여 Actor-Critic 방법을 구현하는 방법을 보여주었습니다.

다음 단계로 Gym의 다른 환경에서 모델의 훈련을 시도할 수 있습니다.

Actor-Critic 방법 및 Cartpole-v0 문제에 대한 추가 정보는 다음 리소스를 참조하세요.

- [Actor-Critic 메서드](https://hal.inria.fr/hal-00840470/document)
- [Actor-Critic 강의(CAL)](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=7&t=0s)
- [Cart Pole 학습 제어 문제 [Barto 등 1983]{/a}](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf)

TensorFlow에서 더 많은 강화 학습 예를 보려면 다음 리소스를 확인하세요.

- [강화 학습 코드 예제(keras.io)](https://keras.io/examples/rl/)
- [TF-Agents 강화 학습 라이브러리](https://www.tensorflow.org/agents)
